# Статистические критерии

In [ ]:
from collections import namedtuple
import scipy.stats as sps
import statsmodels.stats.api as sms
from tqdm.notebook import tqdm as tqdm_notebook
from collections import defaultdict
from statsmodels.stats.proportion import proportion_confint
import numpy as np
import itertools
import seaborn as sns
import scipy.stats as sps
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
sns.set(font_scale=1.5, palette='Set2')
ExperimentComparisonResults = namedtuple('ExperimentComparisonResults',
                                        ['pvalue', 'effect', 'ci_length', 'left_bound', 'right_bound'])

## T-test

In [ ]:
def absolute_ttest(control, test):
    mean_control = np.mean(control)
    mean_test = np.mean(test)
    var_mean_control  = np.var(control) / len(control)
    var_mean_test  = np.var(test) / len(test)

    difference_mean = mean_test - mean_control
    difference_mean_var = var_mean_control + var_mean_test
    difference_distribution = sps.norm(loc=difference_mean, scale=np.sqrt(difference_mean_var))

    left_bound, right_bound = difference_distribution.ppf([0.025, 0.975])
    ci_length = (right_bound - left_bound)
    pvalue = 2 * min(difference_distribution.cdf(0), difference_distribution.sf(0))
    effect = difference_mean
    return ExperimentComparisonResults(pvalue, effect, ci_length, left_bound, right_bound)

In [ ]:
def relative_ttest(control, test):
    mean_control = np.mean(control)
    var_mean_control  = np.var(control) / len(control)

    difference_mean = np.mean(test) - mean_control
    difference_mean_var  = np.var(test) / len(test) + var_mean_control

    covariance = -var_mean_control

    relative_mu = difference_mean / mean_control
    relative_var = difference_mean_var / (mean_control ** 2) \
                    + var_mean_control * ((difference_mean ** 2) / (mean_control ** 4))\
                    - 2 * (difference_mean / (mean_control ** 3)) * covariance
    relative_distribution = sps.norm(loc=relative_mu, scale=np.sqrt(relative_var))
    left_bound, right_bound = relative_distribution.ppf([0.025, 0.975])

    ci_length = (right_bound - left_bound)
    pvalue = 2 * min(relative_distribution.cdf(0), relative_distribution.sf(0))
    effect = relative_mu
    return ExperimentComparisonResults(pvalue, effect, ci_length, left_bound, right_bound)

## Cuped

In [ ]:
def absolute_cuped(control, test, control_before, test_before):
    theta = (np.cov(control, control_before)[0, 1] + np.cov(test, test_before)[0, 1]) /\
                (np.var(control_before) + np.var(test_before))

    control_cup = control - theta * control_before
    test_cup = test - theta * test_before
    return absolute_ttest(control_cup, test_cup)

In [ ]:
def relative_cuped(control, test, control_before, test_before):
    theta = (np.cov(control, control_before)[0, 1] + np.cov(test, test_before)[0, 1]) /\
                (np.var(control_before) + np.var(test_before))

    control_cup = control - theta * control_before
    test_cup = test - theta * test_before

    mean_den = np.mean(control)
    mean_num = np.mean(test_cup) - np.mean(control_cup)
    var_mean_den  = np.var(control) / len(control)
    var_mean_num  = np.var(test_cup) / len(test_cup) + np.var(control_cup) / len(control_cup)

    cov = -np.cov(control_cup, control)[0, 1] / len(control)

    relative_mu = mean_num / mean_den
    relative_var = var_mean_num / (mean_den ** 2)  + var_mean_den * ((mean_num ** 2) / (mean_den ** 4))\
                - 2 * (mean_num / (mean_den ** 3)) * cov

    relative_distribution = sps.norm(loc=relative_mu, scale=np.sqrt(relative_var))
    left_bound, right_bound = relative_distribution.ppf([0.025, 0.975])

    ci_length = (right_bound - left_bound)
    pvalue = 2 * min(relative_distribution.cdf(0), relative_distribution.sf(0))
    effect = relative_mu
    return ExperimentComparisonResults(pvalue, effect, ci_length, left_bound, right_bound)

## Paired stratification

In [ ]:
def paired_ttest(control, test):
    mean_control = np.mean(control)
    mean_test = np.mean(test)

    difference_mean = mean_test - mean_control
    difference_mean_var = np.var(test - control) / len(control)
    difference_distribution = sps.norm(loc=difference_mean, scale=np.sqrt(difference_mean_var))

    left_bound, right_bound = difference_distribution.ppf([0.025, 0.975])
    ci_length = (right_bound - left_bound)
    pvalue = 2 * min(difference_distribution.cdf(0), difference_distribution.sf(0))
    effect = difference_mean
    return ExperimentComparisonResults(pvalue, effect, ci_length, left_bound, right_bound)

In [ ]:
def paired_relative_ttest(control, test):
    mean_control = np.mean(control)
    mean_test = np.mean(test)

    var_mean_control = np.var(control) / len(control)
    difference_mean = mean_test - mean_control
    difference_mean_var  = np.var(test - control) / len(test)

    covariance = -np.cov(test - control, control)[0, 1] / len(control)

    relative_mu = difference_mean / mean_control
    relative_var = difference_mean_var / (mean_control ** 2) \
                    + var_mean_control * ((difference_mean ** 2) / (mean_control ** 4))\
                    - 2 * (difference_mean / (mean_control ** 3)) * covariance
    relative_distribution = sps.norm(loc=relative_mu, scale=np.sqrt(relative_var))
    left_bound, right_bound = relative_distribution.ppf([0.025, 0.975])

    ci_length = (right_bound - left_bound)
    pvalue = 2 * min(relative_distribution.cdf(0), relative_distribution.sf(0))
    effect = relative_mu
    return ExperimentComparisonResults(pvalue, effect, ci_length, left_bound, right_bound)

In [ ]:
def paired_relative_cuped(control, test, control_before, test_before):
    theta = np.cov(test - control, test_before - control_before)[0, 1] /\
                np.var(test_before - control_before)

    control_cup = control - theta * control_before
    test_cup = test - theta * test_before

    mean_den = np.mean(control)
    mean_num = np.mean(test_cup - control_cup)
    var_mean_den  = np.var(control) / len(control)
    var_mean_num  = np.var(test_cup - control_cup) / len(control_cup)

    cov = np.cov(test_cup - control_cup, control)[0, 1] / len(control)

    relative_mu = mean_num / mean_den
    relative_var = var_mean_num / (mean_den ** 2)  + var_mean_den * ((mean_num ** 2) / (mean_den ** 4))\
                - 2 * (mean_num / (mean_den ** 3)) * cov

    relative_distribution = sps.norm(loc=relative_mu, scale=np.sqrt(relative_var))
    left_bound, right_bound = relative_distribution.ppf([0.025, 0.975])

    ci_length = (right_bound - left_bound)
    pvalue = 2 * min(relative_distribution.cdf(0), relative_distribution.sf(0))
    effect = relative_mu
    return ExperimentComparisonResults(pvalue, effect, ci_length, left_bound, right_bound)

In [ ]:
def splitter(before_metrics):
    size = len(before_metrics)

    sorted_array = np.sort(before_metrics)[::-1]
    control = []
    test = []
    for i in range(0, size, 2):
        if np.random.rand() < 0.5:
            control.append(sorted_array[i])
            test.append(sorted_array[i + 1])
        else:
            control.append(sorted_array[i + 1])
            test.append(sorted_array[i])

    return np.array(control), np.array(test)

In [ ]:
def splitter(before_metrics):
    size = len(before_metrics)

    sorted_array = np.sort(before_metrics)[::-1]
    control = []
    test = []
    for i in range(0, size, 2):
        if np.random.rand() < 0.5:
            control.append(sorted_array[i])
            test.append(sorted_array[i + 1])
        else:
            control.append(sorted_array[i + 1])
            test.append(sorted_array[i])

    return np.array(control), np.array(test)

# Загрузка данных

In [ ]:
events = pd.read_parquet('events.pq')
clickstream = pd.read_parquet('clickstream.pq')

result = (
    pd.merge(
        left=clickstream,
        right=events[events['is_contact'] == 1],
        how='inner',
        on='event'
    )
    .assign(week=lambda x: pd.to_datetime(x['event_date']).dt.to_period('W'))
    .groupby(['cookie', 'week'], as_index=False)
    .agg(contacts=('event', 'count'))
    .rename(columns={'week': 'dt'})
)

In [ ]:
result = (
    pd.merge(
        left=clickstream,
        right=events[events['is_contact'] == 1],
        how='inner',
        on='event'
    )
    .assign(week=lambda x: pd.to_datetime(x['event_date']).dt.to_period('W'))
    .groupby(['cookie', 'week'], as_index=False)
    .agg(contacts=('event', 'count'))
    .rename(columns={'week': 'dt'})
)

# Визуализация данных

In [ ]:
threshold = result['contacts'].quantile(0.975)

filtered_data = result[result['contacts'] <= threshold]
dist_df = filtered_data['contacts'].value_counts().sort_index().reset_index()
dist_df.columns = ['contacts', 'unique_users']

plt.figure(figsize=(12, 7))
sns.set_style("whitegrid")
plt.rcParams['font.size'] = 12

ax = sns.barplot(
    x='contacts',
    y='unique_users',
    data=dist_df,
    color='deepskyblue',
    alpha=0.8,
    edgecolor='white',
    linewidth=0.5
)

mean_val = filtered_data['contacts'].mean()
median_val = filtered_data['contacts'].median()

plt.axvline(mean_val, color='crimson', linestyle='--', linewidth=2,
            label=f'Среднее: {mean_val:.1f}')
plt.axvline(median_val, color='navy', linestyle=':', linewidth=2,
            label=f'Медиана: {median_val:.1f}')

plt.title('Распределение количества контактов\n(исключены топ 2.5% выбросов)', fontsize=14, pad=20)
plt.xlabel('Количество контактов', fontsize=12)
plt.ylabel('Количество уникальных пользователей', fontsize=12)

if len(dist_df) > 20:
    plt.xticks(rotation=90, fontsize=10)
    ax.set_xticks(ax.get_xticks()[::5])
else:
    plt.xticks(rotation=45)

plt.legend(frameon=True, fontsize=12)
plt.tight_layout()
plt.savefig('real.pdf')

# Проверка на А/А- и А/Б-тестах

In [ ]:
events = pd.read_parquet('events.pq')
clickstream = pd.read_parquet('clickstream.pq')

result = (
    pd.merge(
        left=clickstream,
        right=events[events['is_contact'] == 1],
        how='inner',
        on='event'
    )
    .assign(week=lambda x: pd.to_datetime(x['event_date']).dt.to_period('W'))
    .groupby(['cookie', 'week'], as_index=False)
    .agg(contacts=('event', 'count'))
    .rename(columns={'week': 'dt'})
)

In [ ]:
result['week'] = result['dt'].astype(str).str.split('/').str[0]
result['user_id'] = result['cookie']
result['revenue'] = result['contacts']
result = result[['user_id', 'week', 'revenue']]

In [ ]:
df = result.copy()

In [ ]:
df['week'] = pd.to_datetime(df['week'])
all_weeks = pd.date_range(start=df['week'].min(), end=df['week'].max(), freq='W-MON')
users = df['user_id'].unique()
full_index = pd.MultiIndex.from_product([users, all_weeks], names=['user_id', 'week'])
df_full = df.set_index(['user_id', 'week']).reindex(full_index, fill_value=0).reset_index()
df_full['week'] = df_full['week'].dt.strftime('%Y-%m-%d')

In [ ]:
def splitter(before_df, revenue_col='before_total'):
    sorted_df = before_df.sort_values(revenue_col, ascending=False).reset_index(drop=True)
    pairs = sorted_df.groupby(sorted_df.index // 2)
    control_users = []
    test_users = []
    for _, pair in pairs:
        if len(pair) == 2:
            if np.random.rand() < 0.5:
                control_users.append(pair.iloc[0]['user_id'])
                test_users.append(pair.iloc[1]['user_id'])
            else:
                control_users.append(pair.iloc[1]['user_id'])
                test_users.append(pair.iloc[0]['user_id'])
        else:
            if np.random.rand() < 0.5:
                control_users.append(pair.iloc[0]['user_id'])
            else:
                test_users.append(pair.iloc[0]['user_id'])
    control_values = before_df.set_index('user_id').loc[control_users][revenue_col].values
    test_values = before_df.set_index('user_id').loc[test_users][revenue_col].values
    return (np.array(control_values), np.array(test_values), np.array(control_users), np.array(test_users))

In [ ]:
user_data = df_full.groupby('user_id').agg(
    before_total=('revenue', lambda x: x.iloc[:-1].sum()),
    test_total=('revenue', lambda x: x.iloc[-1:].sum()),
    before_mean=('revenue', lambda x: x.iloc[:-1].mean()),
    before_median=('revenue', lambda x: x.iloc[:-1].median())
).reset_index()

In [ ]:
user_data_helper = user_data.copy()

In [ ]:
user_data = user_data_helper.copy()

In [ ]:
# user_data = user_data[user_data['before_total'] >= 10]

In [ ]:
user_data = user_data.sample(n=5000).reset_index()
user_data = user_data.drop('index', axis = 1)

In [ ]:
bad_cnt_ttest = 0
bad_cnt_ttest_c = 0
bad_cnt_paired = 0
bad_cnt_paired_mean = 0
bad_cnt_paired_median = 0
power_ttest = 0
power_paired = 0
power_paired_mean = 0
power_paired_median = 0
N = 1000
a = 0

for i in tqdm_notebook(range(N)):
    shuffled = user_data.sample(frac=1).reset_index(drop=True)
    split_idx = len(shuffled) // 2
    control_group = shuffled.iloc[:split_idx]
    test_group = shuffled.iloc[split_idx:]
    C_ttest = control_group['test_total'].values
    T_ttest = test_group['test_total'].values * (1 + a)

    C_b_paired, T_b_paired, C_ids, T_ids = splitter(user_data[['user_id', 'before_total']], 'before_total')
    C_paired = user_data.set_index('user_id').loc[C_ids]['test_total'].values
    T_paired = user_data.set_index('user_id').loc[T_ids]['test_total'].values * (1 + a)

    C_b_paired_mean, T_b_paired_mean, C_ids_mean, T_ids_mean = splitter(user_data[['user_id', 'before_mean']], 'before_mean')
    C_paired_mean = user_data.set_index('user_id').loc[C_ids_mean]['test_total'].values
    T_paired_mean = user_data.set_index('user_id').loc[T_ids_mean]['test_total'].values * (1 + a)

    C_b_paired_median, T_b_paired_median, C_ids_median, T_ids_median = splitter(user_data[['user_id', 'before_median']], 'before_median')
    C_paired_median = user_data.set_index('user_id').loc[C_ids_median]['test_total'].values
    T_paired_median = user_data.set_index('user_id').loc[T_ids_median]['test_total'].values * (1 + a)

    _, _, _, left_t, right_t = relative_ttest(C_ttest, T_ttest)
    _, _, _, left_t_c, right_t_c = relative_cuped(C_ttest, T_ttest, control_group['before_total'].values, test_group['before_total'].values)
    _, _, _, left_p, right_p = paired_relative_ttest(C_paired, T_paired)
    _, _, _, left_pm, right_pm = paired_relative_ttest(C_paired_mean, T_paired_mean)
    _, _, _, left_pmd, right_pmd = paired_relative_ttest(C_paired_median, T_paired_median)

    bad_cnt_ttest += int(left_t > a or right_t < a)
    bad_cnt_ttest_c += int(left_t_c > a or right_t_c < a)
    bad_cnt_paired += int(left_p > a or right_p < a)
    bad_cnt_paired_mean += int(left_pm > a or right_pm < a)
    bad_cnt_paired_median += int(left_pmd > a or right_pmd < a)

    power_ttest += int(left_t > 0)
    power_paired += int(left_p > 0)
    power_paired_mean += int(left_pm > 0)
    power_paired_median += int(left_pmd > 0)

left_real_level_ttest, right_real_level_ttest = proportion_confint(bad_cnt_ttest, N, alpha=0.05, method='wilson')
left_real_level_ttest_c, right_real_level_ttest_c = proportion_confint(bad_cnt_ttest_c, N, alpha=0.05, method='wilson')
left_real_level_paired, right_real_level_paired = proportion_confint(bad_cnt_paired, N, alpha=0.05, method='wilson')
left_real_level_paired_mean, right_real_level_paired_mean = proportion_confint(bad_cnt_paired_mean, N, alpha=0.05, method='wilson')
left_real_level_paired_median, right_real_level_paired_median = proportion_confint(bad_cnt_paired_median, N, alpha=0.05, method='wilson')
left_power_ttest, right_power_ttest = proportion_confint(power_ttest, N, alpha=0.05, method='wilson')
left_power_paired, right_power_paired = proportion_confint(power_paired, N, alpha=0.05, method='wilson')
left_power_paired_mean, right_power_paired_mean = proportion_confint(power_paired_mean, N, alpha=0.05, method='wilson')
left_power_paired_median, right_power_paired_median = proportion_confint(power_paired_median, N, alpha=0.05, method='wilson')

print(f"Alpha, t test: {round(bad_cnt_ttest / N, 4)}; CI AA, t test: [{round(left_real_level_ttest, 5)}, {round(right_real_level_ttest, 5)}]")
print(f"Alpha, t test c: {round(bad_cnt_ttest_c / N, 4)}; CI AA, t test c: [{round(left_real_level_ttest_c, 5)}, {round(right_real_level_ttest_c, 5)}]")
print(f"Alpha, paired: {round(bad_cnt_paired / N, 4)}; CI AA, paired: [{round(left_real_level_paired, 5)}, {round(right_real_level_paired, 5)}]")
print(f"Alpha, paired mean: {round(bad_cnt_paired_mean / N, 4)}; CI AA, paired mean: [{round(left_real_level_paired_mean, 5)}, {round(right_real_level_paired_mean, 5)}]")
print(f"Alpha, paired median: {round(bad_cnt_paired_median / N, 4)}; CI AA, paired median: [{round(left_real_level_paired_median, 5)}, {round(right_real_level_paired_median, 5)}]")
print(f"1-Beta, t test: {round(power_ttest / N, 4)}; CI AB, t test: [{round(left_power_ttest, 5)}, {round(right_power_ttest, 5)}]")
print(f"1-Beta, paired: {round(power_paired / N, 4)}; CI AB, paired: [{round(left_power_paired, 5)}, {round(right_power_paired, 5)}]")
print(f"1-Beta, paired mean: {round(power_paired_mean / N, 4)}; CI AB, paired mean: [{round(left_power_paired_mean, 5)}, {round(right_power_paired_mean, 5)}]")
print(f"1-Beta, paired median: {round(power_paired_median / N, 4)}; CI AB, paired median: [{round(left_power_paired_median, 5)}, {round(right_power_paired_median, 5)}]")